In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import json
from selenium.common.exceptions import NoSuchElementException

In [19]:
# 로그인을 위해 json 파일로 저장한 id와 pwd를 불러온다
with open("accountNaver.json", 'r') as f:
    account = json.load(f)

In [60]:
#크롬 브라우저를 이용하여 webdriver 객체를 만든다
driver_path = 'chromedriver.exe'
driver =webdriver.Chrome(driver_path)

In [61]:
#driver 객체를 통해  url에 request한다
driver.get("https://www.naver.com")

In [48]:
# 로그인 form의 id와 password에 json으로 불러온 계정정보를 send한다
driver.find_element_by_id('id').send_keys(account['id'])
driver.find_element_by_id('pw').send_keys(account['pwd'])

In [49]:
# 로그인 버튼을 찾아 클릭한다
driver.find_element_by_css_selector(".btn_login > input").click()

로그인이 되었는데

네이버가 새로운 기기로 인식하여 기기 등록이 필요한 경우가 있다

이 상황을 위한 예외처리가 필요하다

In [50]:
#driver.find_element_by_css_selector(".btn_upload > a").click() #기기등록
#driver.find_element_by_css_selector(".btn_maintain > a").click() #로그인유지

In [62]:
#네이버 뉴스로 이동
driver.get("https://news.naver.com/")

In [70]:
#가장 많이 본 정치 뉴스를 선택
driver.find_element_by_id("right.ranking_tab_100").click()

In [71]:
#뉴스로 연결되는 링크가 담긴 태그를 가져온다
aList = driver.find_elements_by_css_selector(".section_list_ranking > li > a")

In [72]:
#링크인 href 어트리뷰트를 가져올수있다
hrefList = [a.get_attribute('href') for a in aList]

In [75]:
hrefList[2]

'https://news.naver.com/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=449&aid=0000155623&date=20180719&type=2&rankingSeq=3&rankingSectionId=100'

In [34]:
#뉴스 기사 제목은 다음과 같다
for a in aList:
     if len(a.text)>1:
        print(a.text)

해병대 "영결식 등 장례절차 유가족 의견 따를 것"
김병준, 文정부 "국가주의" 언급에 한병도 "안 맞는 표현"
청와대, 성폭력 무고죄 수사중단 “헌법 위반 아니다”
장제원, 강원랜드 수사외압 불기소 결론에 法장관 질타
검찰, 김무성 딸 시아버지 회사 허위 취업 의혹 수사(종합)
국방부, 軍 정치개입 양심선언 장교 파면처분 취소
법무부 장관에 호통친 이은재 "왜 다들 민변 출신이냐?"
민주 당권주자 대진표 완성…이해찬은 '장고중'
<구멍난 대북제재>작년 北석탄 싣고 韓 들어왔던 배 2척 이후로도 22차례 자유롭게 ‘들락날락’
최재성·김두관, 민주당 대표 경선 출마 선언


In [35]:
# 클릭을 통해서도 이동할 수 있다
aList[0].click()

In [39]:
# 네이버 뉴스 댓글창으로 이동
driver.find_element_by_class_name("simplecmt_link_text").click()

In [42]:
# 작성위해 댓글란에 댓글 내용 send
driver.find_element_by_class_name("u_cbox_text").send_keys("댓글")

In [43]:
# 등록 버튼 클릭
driver.find_element_by_class_name("u_cbox_btn_upload").click()

![1](https://user-images.githubusercontent.com/38183218/42927139-7d67aabe-8b6e-11e8-8bf3-c52724d57332.PNG)

댓글이 등록되었다!

In [78]:
#본격적인 댓글 bot 구현을 위해 함수로 만들어보자
def naver_comment_bot(jsonFileName:str,section:str,comment:str):
    try: 
        with open(jsonFileName+".json", 'r') as f:
            account = json.load(f)
    
        driver_path = 'chromedriver.exe'
        driver =webdriver.Chrome(driver_path)
        driver.get("https://www.naver.com")
        
        driver.find_element_by_id('id').send_keys(account['id'])
        driver.find_element_by_id('pw').send_keys(account['pwd'])
        driver.find_element_by_css_selector(".btn_login > input").click()
        
        # 기기등록이 필요한 경우 활성화
        #driver.find_element_by_css_selector(".btn_upload > a").click() 
        #driver.find_element_by_css_selector(".btn_maintain > a").click()

        driver.get("https://news.naver.com/")
        
        #뉴스 섹션을 선택하게 하였다
        sections = ['정치','경제','사회','생활/문화','세계','IT/과학']
        if section in sections:
            driver.find_element_by_id("right.ranking_tab_10"+str(sections.index(section))).click()
            
        else:
            print("section을 다시 확인하세요")
            return
        
        #가장 많이 본 뉴스 top10 리스트
        aList = driver.find_elements_by_css_selector(".section_list_ranking > li > a")
        hrefList = [a.get_attribute('href') for a in aList]
            
        #주어진 기사 리스트를 돌며 모든 기사에 같은댓글을 남긴다
        for href in hrefList:
            if len(href)>1:
                driver.get(href)
                driver.implicitly_wait(1)# url 요청후 로딩 시간 걸리므로 wait넣어준다
                    
            # 네이버 댓글창으로 이동 이라는 버튼이 존재할 경우 클릭해야한다
            try:
                driver.find_element_by_class_name("simplecmt_link_text").click()
                driver.implicitly_wait(1)
                
                driver.find_element_by_class_name("u_cbox_text").send_keys(comment)
                driver.find_element_by_class_name("u_cbox_btn_upload").click()
    
            except NoSuchElementException as e0:
                driver.find_element_by_class_name("u_cbox_text").send_keys(comment)
                driver.find_element_by_class_name("u_cbox_btn_upload").click()
                continue
            except Exception as e:
                print(e)
                continue
                
    except FileNotFoundError as e1:
        print("필요한 파일을 확인하세요")
    
    except NoSuchElementException as e3:
        print(e3)

    except Exception as e:
        print(e)

In [82]:
naver_comment_bot("accountNaver","경제","한국 경제 화이팅입니다!!")

Alert Text: None
Message: unexpected alert open: {Alert text : 오류가 발생했습니다.}
  (Session info: chrome=67.0.3396.99)
  (Driver info: chromedriver=2.40.565498 (ea082db3280dd6843ebfb08a625e3eb905c4f5ab),platform=Windows NT 10.0.17134 x86_64)

Alert Text: None
Message: unexpected alert open: {Alert text : 오류가 발생했습니다.}
  (Session info: chrome=67.0.3396.99)
  (Driver info: chromedriver=2.40.565498 (ea082db3280dd6843ebfb08a625e3eb905c4f5ab),platform=Windows NT 10.0.17134 x86_64)



선플을 남길 생각에 기쁜 마음으로 함수를 실행시켰습니다만 반복적인 댓글을 달지못하게 막혀있습니다ㅜ

코드에는 문제가 없으나 쉽지않네요 역시

![](https://user-images.githubusercontent.com/38183218/42956239-6fe241a0-8bba-11e8-9db2-f6bc0eea5205.PNG)

![](https://user-images.githubusercontent.com/38183218/42956240-700c0d5a-8bba-11e8-85c4-4f8b7ed756da.PNG)

![](https://user-images.githubusercontent.com/38183218/42956644-805ede20-8bbb-11e8-99ce-aef7cd864ecc.PNG)

몇개 달지도 못했는데 그와중에 들통나 버렸네요